In [22]:
import os
import json
import csv
from glob import glob

# Define the directory and file pattern
path = "F:/setup files/downloads/Result1/"
file_pattern = os.path.join(path, "*.json")

# List of files matching our pattern
files = glob(file_pattern)

# Prepare a list to store results
results = []

# Loop through each JSON file
for filename in files:
    # Load JSON data
    with open(filename, 'r') as f:
        data = json.load(f)

    # Determine number of objectives using the first algorithm/generation/solution
    # (Assuming at least one algorithm, one generation, and one individual exist)
    sample_alg = next(iter(data))
    sample_gen = next(iter(data[sample_alg]))
    sample_solution = data[sample_alg][sample_gen][0]
    objectives_count = len(sample_solution)

    # Initialize nadir point with very small values
    nadir = [-float('inf')] * objectives_count

    # Iterate over all algorithms, generations, and individuals to compute nadir
    for algorithm in data:
        for generation in range(1,100):
            # print(algorithm, generation)
            for individual in data[algorithm][str(generation)]:
                # Update nadir for each objective dimension
                for idx, value in enumerate(individual):
                    if value > nadir[idx]:
                        nadir[idx] = value

    # Store filename (or parsed info) and computed nadir point
    base_filename = os.path.basename(filename)
    result_row = {"file": base_filename}
    # Add each objective's nadir value to the row
    for j in range(objectives_count):
        result_row[f"obj_{j+1}_nadir"] = nadir[j]

    results.append(result_row)

# Define CSV fieldnames based on objectives count
fieldnames = ["file"] + [f"obj_{j+1}_nadir" for j in range(objectives_count)]

# Write results to a CSV file
output_csv = os.path.join(path, "nadir_points.csv")
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print(f"Nadir points saved to {output_csv}")

Nadir points saved to F:/setup files/downloads/Result1/nadir_points.csv


In [24]:
import os
import json
import pandas as pd
from pymoo.indicators.igd import IGD
import numpy as np

def cal_igd(front, ref_pareto):
    """
    Calculate the Inverted Generational Distance (IGD) between a given front and a reference Pareto front.
    """
    igd_indicator = IGD(ref_pareto)
    return igd_indicator(front)

# Paths and file info
path = "F:/setup files/downloads/Result1/"
nadir_csv_path = os.path.join(path, "nadir_points.csv")
igd_csv_path = os.path.join(path, "igd_values.csv")
pivot_csv_path = os.path.join(path, "igd_pivot.csv")  # For pivoted result

# Load the nadir points CSV
nadir_df = pd.read_csv(nadir_csv_path)
print("Nadir Points DataFrame:")
print(nadir_df.head())

igd_results = []

# Iterate over each row (each file) in the nadir DataFrame
for index, row in nadir_df.iterrows():
    file_name = row['file']
    file_path = os.path.join(path, file_name)

    # Check if JSON file exists
    if not os.path.isfile(file_path):
        print(f"File {file_name} not found. Skipping.")
        continue

    # Load JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract the nadir point for scaling (used here as a proxy for a reference Pareto front)
    obj_columns = sorted(
        [col for col in nadir_df.columns if col.startswith("obj_") and col.endswith("_nadir")],
        key=lambda x: int(x.split('_')[1])
    )
    # Using the nadir point as the reference Pareto front (this is a simplification).
    # Replace with an actual Pareto front if available.
    ref_pareto = np.array([row[col] for col in obj_columns]).reshape(1, -1)

    # For each algorithm in the JSON data

    last = "100"
    for algorithm in data:
        if last not in data[algorithm]:
            print(f"Generation {last} not found for algorithm {algorithm} in file {file_name}. Skipping.")
            continue

        solutions = data[algorithm][last]
        solutions_np = np.array(solutions)

        if solutions_np.size == 0:
            print(f"No solutions found for algorithm {algorithm} in generation {last} of file {file_name}. Skipping.")
            continue

        try:
            igd_value = cal_igd(solutions_np, ref_pareto)
        except Exception as e:
            print(f"Error computing IGD for algorithm {algorithm} in file {file_name}: {e}")
            continue

        igd_results.append({
            "file": file_name,
            "algorithm": algorithm,
            "IGD": igd_value
        })

# Convert IGD results to a DataFrame
igd_df = pd.DataFrame(igd_results)

# Pivot the DataFrame: rows as file names, columns as algorithms, values as IGD
igd_pivot_df = igd_df.pivot(index='file', columns='algorithm', values='IGD')

# Save the pivoted IGD DataFrame to a CSV file
igd_pivot_df.to_csv(pivot_csv_path)

print(f"Pivoted IGD table saved to {pivot_csv_path}")

Nadir Points DataFrame:
              file  obj_1_nadir  obj_2_nadir  obj_3_nadir
0     example.json     0.861599     0.596660     1.147920
1  LC1_2_10_0.json  1845.818031    37.011668   200.679342
2   LC1_2_1_0.json  1932.163178    42.935566   275.351020
3   LC1_2_2_0.json  1885.601227    34.704041   208.757128
4   LC1_2_3_0.json  1856.797988    39.450236   268.753980
Pivoted hypervolume table saved to F:/setup files/downloads/Result1/hv100_pivot.csv
